## Topics

### In-context learning
### Promt-engineering
### Langchain
### 

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages([
    ("system", "Y aou are a helpful assistant."),
    ("human", "{question}")
])

chain = prompt | llm
response = chain.invoke({"question": "Explain LangChain in one sentence."})
print(response.content)


## Langchain vs LCEL(Langchain Expression Language)

Good question — this is a **common point of confusion**.

### Short answer

**LCEL is part of LangChain.**
It’s **not an alternative**.

> **LangChain = framework**
> **LCEL (LangChain Expression Language) = the modern way to use LangChain**

---

## What is LCEL?

**LCEL** is a **declarative syntax** introduced by LangChain to build chains using **pipe-style composition**.

```python
chain = prompt | llm | output_parser
```

It replaces older, verbose `Chain` classes.

---

## LangChain *before* LCEL (old style)

```python
from langchain.chains import LLMChain

chain = LLMChain(
    llm=llm,
    prompt=prompt
)

result = chain.run("What is LCEL?")
```

❌ More boilerplate
❌ Harder to compose
❌ Less flexible

---

## LangChain *with* LCEL (modern style)

```python
chain = prompt | llm
result = chain.invoke({"question": "What is LCEL?"})
```

✅ Simple
✅ Composable
✅ Streamable
✅ Async-friendly

---

## Key differences

| Feature        | Old LangChain | LCEL             |
| -------------- | ------------- | ---------------- |
| Syntax         | Class-based   | Pipe-based       |
| Composability  | Limited       | Excellent        |
| Streaming      | Hard          | Built-in         |
| Async support  | Inconsistent  | Native           |
| Debugging      | Hard          | Tracing-friendly |
| Future support | Deprecated    | **Recommended**  |

---

## Why LangChain moved to LCEL

* Reduce abstraction complexity
* Make chains **feel like functional pipelines**
* Support streaming, retries, fallbacks
* Make evaluation & tracing easier (LangSmith)

---

## Example: RAG with LCEL

```python
rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
)
```

This would have been **very complex** pre-LCEL.

---

## Mental model

Think of LCEL like:

* **Unix pipes** (`|`)
* **React composition**
* **Functional programming for LLMs**

---

## Should you “learn LangChain or LCEL”?

✔️ Learn **LangChain concepts**
✔️ Use **LCEL syntax exclusively**

If you see tutorials not using `|`, they’re **outdated**.

---

## When NOT to use LCEL

* If you’re not using LangChain at all
* If your app is just **one prompt → one response**

---

If you want, I can:

* Convert **old LangChain code → LCEL**
* Show **LCEL vs raw OpenAI SDK**
* Build a **real RAG pipeline in LCEL**

Just tell me what you want to build.


In **LCEL (LangChain Expression Language)**, a **Runnable** is the *core abstraction* that represents **anything that can be executed** in a LangChain pipeline.

Think of a **Runnable** as a **callable building block** that:

* takes an input
* does *something*
* produces an output
  …and can be **composed**, **chained**, **streamed**, and **parallelized**.

---

## Simple definition

> **A Runnable is an object that implements a standard execution interface (`invoke`, `batch`, `stream`, etc.) so it can be composed with other Runnables in LCEL.**

---

## Why Runnables exist

Before LCEL, LangChain used many different abstractions (`Chain`, `LLMChain`, `Tool`, etc.).
LCEL unifies everything under **one execution model** → **Runnable**.

So in LCEL:

* prompts are Runnables
* models are Runnables
* parsers are Runnables
* chains are Runnables
* *your own Python functions* can be Runnables

---

## Core Runnable interface

Most Runnables support:

```python
runnable.invoke(input)     # single input → single output
runnable.batch(inputs)     # list of inputs → list of outputs
runnable.stream(input)     # streaming output
```

---

## Common Runnable types

Here are the most important ones you'll see in LCEL 👇

### 1. PromptTemplate (Runnable)

```python
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Tell me a joke about {topic}"
)
```

✔ Input: `{"topic": "cats"}`
✔ Output: formatted prompt

---

### 2. LLM / Chat Model (Runnable)

```python
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
```

✔ Input: prompt
✔ Output: model response

---

### 3. Output Parser (Runnable)

```python
from langchain.output_parsers import StrOutputParser

parser = StrOutputParser()
```

✔ Input: model message
✔ Output: clean string

---

### 4. RunnableLambda (wrap a Python function)

```python
from langchain_core.runnables import RunnableLambda

uppercase = RunnableLambda(lambda x: x.upper())
```

✔ Turns any function into a Runnable

---

### 5. RunnableSequence (created with `|`)

This is LCEL magic ✨

```python
chain = prompt | llm | parser
```

This **entire chain is itself a Runnable**.

---

## Runnable composition (LCEL operators)

### Pipe (`|`) → sequence

```python
chain = prompt | llm | parser
```

### Parallel execution

```python
from langchain_core.runnables import RunnableParallel

parallel = RunnableParallel(
    joke=prompt | llm,
    summary=prompt | llm
)
```

### Assign / enrich data

```python
chain = (
    RunnableLambda(lambda x: {"topic": x})
    .assign(result=prompt | llm | parser)
)
```

---

## Key idea to remember

> **Everything in LCEL is a Runnable — and every Runnable can be combined with others using simple Python syntax.**

This makes LCEL:

* declarative
* readable
* composable
* async & streaming friendly

---

If you want, next we can:

* compare **Runnable vs Chain** (old vs new)
* build a **full LCEL app**
* explain **LCEL execution flow step-by-step**
* show **real-world GenAI use cases with Runnables**

Just tell me 🔥


Yes — **you can build agents using LangChain**. In fact, LangChain is one of the most popular frameworks right now for creating **intelligent agents** that can reason, plan, and act in an environment using large language models (LLMs).

Here’s a clear overview to help you understand how this works and how to build them.

---

## 🧠 What Is an Agent in LangChain?

In LangChain, an **agent** is a system that:

1. **Observes** user input
2. **Decides** what tools or actions to use (search, calculator, APIs, etc.)
3. **Executes** steps using those tools
4. **Responds** intelligently to the user

This is more than just a single LLM call — agents can involve **reasoning, planning, and dynamic tool usage**.

---

## 🛠️ Core Components of a LangChain Agent

1. **LLM (Language Model)**
   The model that reasons about what to do next (e.g., GPT-4.x).

2. **Tools**
   These are functions or APIs that the agent can call — e.g., a web search API, math solver, database query tool, browser, or vector store retriever.

3. **Agent Logic**
   Controls how the agent chooses which tool to use when.

4. **Memory (optional)**
   Allows the agent to remember context across interactions.

---

## 🚀 Simple Agent Example (Python)

Here’s a minimal example using LangChain Python:

```python
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI

# 1) Create an LLM
llm = OpenAI(temperature=0)

# 2) Define a tool (for example, a simple calculator)
def calculator_fn(input: str):
    try:
        result = eval(input)
        return str(result)
    except Exception as e:
        return "Error"

calculator_tool = Tool(
    name="calculator",
    func=calculator_fn,
    description="Useful for math calculations"
)

# 3) Initialize the agent
agent = initialize_agent(
    tools=[calculator_tool],
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

# 4) Run the agent
response = agent.run("What is 123 * 45?")
print(response)
```

This agent will decide to call the `calculator` tool when it recognizes a math request.

---

## 🧩 More Advanced Agent Features

### 🔍 1. Tool-Enhanced Reasoning

Agents can be connected to:

* Web search tools
* Browsers (for live search)
* Databases
* Knowledge bases or vector search systems
* APIs like weather, finance, or custom REST endpoints

### 🧠 2. Memory Integration

Agents can **remember facts** across messages:

* Personal preferences
* Ongoing tasks
* Context from earlier conversations

Example: Remembering a user’s project details.

### 🗂️ 3. Retrieval-Augmented Generation (RAG)

Agents can fetch relevant documents from:

* Vector stores (e.g., Pinecone, Chroma)
* Local files (PDFs, text)
* Databases

Then combine that with reasoning to answer complex queries.

---

## 📈 Real-World Use Cases

Here are practical agent use cases across industries:

| Industry         | Example Agent Use                                        |
| ---------------- | -------------------------------------------------------- |
| Customer Support | Auto-triage tickets + fetch solutions from KB            |
| Finance          | Answer live stock questions using market data APIs       |
| Legal            | Summarize and query contracts stored in vector DB        |
| Healthcare       | Retrieve patient data + suggest next steps (with safety) |
| Dev Tools        | Code generation + testing + API calling                  |

**Example:**
A legal agent reads a contract PDF, answers questions about clauses, and highlights risks — all with sources referenced.

---

## 🧠 Choosing an Agent Type

LangChain supports several agent styles:

* **Zero-Shot React** – the agent decides actions based on instructions
* **Structured Reactions** – agents follow a template
* **Conversation Agents** – retain chat context
* **Planner + Executor** – breaks tasks into steps

---

## 🚀 When to Use an Agent vs. a Simple Chain

| Use Case                                  | Simple Chain | Agent |
| ----------------------------------------- | ------------ | ----- |
| Straightforward text generation           | ✅            | ❌     |
| One-off summarization or translation      | ✅            | ❌     |
| Tasks needing external calls or reasoning | ❌            | ✅     |
| Iterative multi-step decision making      | ❌            | ✅     |

---

## 🧩 Tips for Better Agents

✨ **Start small** — add simple tools, then expand
📍 **Use logging/verbose mode** to debug decisions
🔐 **Sandbox risky tools** (e.g., eval)
🧠 **Tune memory** to avoid clutter

---

If you want, I can show you **how to build a specific type of agent** (e.g., web-enabled, database-connected, or multi-step planner). Just tell me your use case!

